# Data preparation 

In [26]:
import numpy as np
import pandas as pd
import os.path as pt
import re
from io import StringIO

In [27]:
class KnnCpgFeatureExtractor(object):
    """Extracts k CpG sites next to target sites. Excludes CpG sites at the
    same position.
    """

    def __init__(self, k=1):
        self.k = k

        t = [''] * 4 * k
        for i in range(k):
            t[k - 1 - i] = 'cpg_l_%d' % (i + 1)
            t[k + i] = 'cpg_r_%d' % (i + 1)
            t[3 * k - 1 - i] = 'dist_l_%d' % (i + 1)
            t[3 * k + i] = 'dist_r_%d' % (i + 1)
        self.labels = t

    def extract(self, x, y, ys):
        """Extracts state and distance of k CpG sites next to target sites.
        Target site is excluded.

        Parameters
        ----------
        x: numpy array with target positions sorted in ascending order
        y: numpy array with source positions sorted in ascending order
        ys: numpy array with source CpG states

        Returns
        -------
        numpy array with len(x) rows and 4*k columns:
            0:k     CpG states left side
            k:2k    CpG states right side
            2k:3k   Distances left side
            3k:4k   Distances right side
        """

        n = len(x)
        m = len(y)
        k = self.k
        kk = 2 * self.k
        yc = self.__larger_equal(x, y)
        rv = np.empty((n, 4 * k))
        rv.fill(np.nan)
        for i in range(n):
            # Left side
            yl = yc[i] - k
            yr = yc[i] - 1
            if yr >= 0:
                xl = 0
                xr = k - 1
                if yl < 0:
                    xl += np.abs(yl)
                    yl = 0
                xr += 1
                yr += 1
                # CpG states
                rv[i, xl:xr] = ys[yl:yr]
                xl += kk
                xr += kk
                # Distance
                rv[i, xl:xr] = np.abs(y[yl:yr] - x[i])

            # Right side
            yl = yc[i]
            if yl >= m:
                continue
            if x[i] == y[yl]:
                yl += 1
                if yl >= m:
                    continue
            yr = yl + k - 1
            xl = 0
            xr = k - 1
            if yr >= m:
                xr -= yr - m + 1
                yr = m - 1
            xl += k
            xr += k + 1
            yr += 1
            # CpG state
            rv[i, xl:xr] = ys[yl:yr]
            xl += kk
            xr += kk
            # Distance
            rv[i, xl:xr] = np.abs(y[yl:yr] - x[i])
        return rv

    def __larger_equal(self, x, y):
        """Returns for each x[i] index j, s.t. y[j] >= x[i].

        Parameters
        ----------
        x : numpy array of with positions sorted in ascending order
        y : numpy array of with positions sorted in ascending order
        """

        n = len(x)
        m = len(y)
        rv = np.empty(n, dtype=np.int)
        i = 0
        j = 0
        while i < n and j < m:
            while j < m and x[i] > y[j]:
                j += 1
            rv[i] = j
            i += 1
        if i < n:
            # x[i] > y[m - 1]
            rv[i:] = m
        return rv
    
    
class IntervalFeatureExtractor(object):
    """Checks if positions are in a list of intervals (start, end)."""

    @staticmethod
    def join_intervals(s, e):
        """Transforms a list of possible overlapping intervals into
        non-overlapping intervals.

        Parameters
        ----------
        s : list with start of interval sorted in ascending order
        e : list with end of interval

        Returns
        -------
        Tuple (s, e) of non-overlapping intervals
        """

        rs = []
        re = []
        n = len(s)
        if n == 0:
            return (rs, re)
        l = s[0]
        r = e[0]
        for i in range(1, n):
            if s[i] > r:
                rs.append(l)
                re.append(r)
                l = s[i]
                r = e[i]
            else:
                r = max(r, e[i])
        rs.append(l)
        re.append(r)
        return (rs, re)

    @staticmethod
    def index_intervals(x, ys, ye):
        """Returns for positions x[i] index j, s.t. ys[j] <= x[i] <= ye[j] or -1.
           Intervals must be non-overlapping!

        Parameters
        ----------
        x : list of positions
        ys: list with start of interval sorted in ascending order
        ye: list with end of interval

        Returns
        -------
        numpy array of same length than x with index or -1
        """

        n = len(ys)
        m = len(x)
        rv = np.empty(m, dtype=np.int)
        rv.fill(-1)
        i = 0
        j = 0
        while i < n and j < m:
            while j < m and x[j] <= ye[i]:
                if x[j] >= ys[i]:
                    rv[j] = i
                j += 1
            i += 1
        return rv

    def extract(self, x, ys, ye):
        return self.index_intervals(x, ys, ye) >= 0

In [28]:
def format_chromo(chromo):
    chromo = str(chromo)
    chromo = chromo.upper()
    chromo = re.sub('^CHR', '', chromo)
    return chromo

def format_chromos(chromos):
    return [format_chromo(x) for x in chromos]

def read_txt(filename, nrows=None):
    d = pd.read_table(filename)
    sample = d.columns[-1]
    d = d.iloc[:, [0, 1, 3]]
    d.columns = ['chromo', 'pos', 'value']
    d['sample'] = sample
    d['chromo'] = format_chromos(d['chromo'])
    return d

def read_txts(filenames):
    return pd.concat([read_txt(f) for f in filenames])

def read_bed(filename):
    d = pd.read_table(filename, header=None)
    d = d.iloc[:, :3]
    d.columns = ['chromo', 'start', 'end']
    d['chromo'] = format_chromos(d['chromo'])
    return d

def read_annos(filenames):
    anno_tables = {}
    for anno_file in filenames:
        anno_table = read_bed(anno_file)
        anno_table = anno_table.sort(['chromo', 'start'])
        anno_name = pt.splitext(pt.basename(anno_file))[0]
        anno_tables[anno_name] = anno_table
    return anno_tables

def spread(d):
    return pd.pivot_table(d, index=['chromo', 'pos'], columns='sample', values='value')

def gather(d):
    return pd.melt(d.reset_index(), id_vars=['chromo', 'pos'], var_name='sample', value_name='value')

def is_int(d):
    return d == round(d)

def size_to_int(size, n):
    if is_int(size):
        return size
    else:
        return round(size * n)
    
def train_test(d, test_size=0.5):
    n = d.shape[0]
    test_size = size_to_int(test_size, n)
    idx = np.arange(n)
    test_idx = np.random.choice(idx, test_size, replace=False)
    test_idx = np.in1d(idx, test_idx)
    train = d.loc[~test_idx]
    test = d.loc[test_idx]
    return (train, test)

def train_test_eval(d, test_size=0.5, eval_size=0.1):
    h, test = train_test(d, test_size)
    train, eval_ = train_test(h, eval_size)
    return (train, test, eval_)

def group_apply(d, by, fun, *args, **kwargs):
    g = d.groupby(by)
    r_all = []
    for k in g.groups.keys():
        dg = g.get_group(k)
        r = fun(dg, *args, **kwargs)
        if type(by) is list:
            for i in range(len(by)):
                r[by[i]] = k[i]
        else:
            r[by] = k
        r_all.append(r)
    r_all = pd.concat(r_all)
    return r_all

def get_pos(d):
    return pd.pivot_table(d, index=['chromo', 'pos'], columns='sample', values='value').reset_index()['pos'].values
    

def knn_features_sample(d, fe, pos):
    f = fe.extract(pos, d['pos'].values, d['value'].values)
    f = pd.DataFrame(f, columns=fe.labels)
    f['pos'] = pos
    f = pd.melt(f, id_vars='pos', var_name='feature', value_name='value')
    return f
        
def knn_features_chromo(d, fe):
    pos = get_pos(d)
    return group_apply(d, 'sample', knn_features_sample, fe, pos)
           
def knn_features(d, k=5):
    fe = KnnCpgFeatureExtractor(k)
    return group_apply(d, 'chromo', knn_features_chromo, fe)

def anno_features_chromo(d, annos, fe):
    pos = get_pos(d)
    f_all = []
    for anno_name, anno_table in annos.items():
        start, end = fe.join_intervals(anno_table['start'].values, anno_table['end'].values)
        f = fe.extract(pos, start, end)
        f = pd.DataFrame(dict(pos=pos, feature=anno_name, value=f))
        f_all.append(f)
    f_all = pd.concat(f_all)
    return f_all

def anno_features(d, annos):
    fe = IntervalFeatureExtractor()
    return group_apply(d, 'chromo', anno_features_chromo, annos, fe)

def features(d, k=5, annos=None):
    f_all = []
    f = d.copy()
    f['feature'] = 'cpg'
    f_all.append(f)
    if k is not None:
        f = knn_features(d, k)
        f['feature'] = ['knn_' + x for x in f['feature']]
        f_all.append(f)
        
    if annos is not None:
        f = anno_features(d, annos)
        f['sample'] = 'global'
        f['feature'] = ['anno_' + x for x in f['feature']]
        f_all.append(f)
    f_all = pd.concat(f_all)
    return f_all

In [224]:
s = '''chromo pos s1 s2
1 1 1 NA
1 2 NA 0
1 3 1 0
2 1 0 0
2 3 1 0
'''
dcpg = pd.read_table(StringIO(s), sep=' ')
dt = dict()
dt['cpg'] = pd.melt(dcpg, id_vars=['chromo', 'pos'], var_name='sample', value_name='value').dropna()

In [29]:
d = dict()
d['cpg'] = read_txts(['txt/CSC2_3B.txt', 'txt/CSC2_3C.txt', 'txt/CSC4_7B.txt'])
d['annos'] = read_annos(['annos/gene_body.bed', 'annos/LMRs.bed'])

In [30]:
dc = train_test_eval(d['cpg'])
df = [features(x, k=5, annos=d['annos']) for x in dc]

In [34]:
df[0].head()

,chromo,feature,pos,sample,value
0,1,cpg,3003339,CSC2_3B,1
3,1,cpg,3014601,CSC2_3B,1
4,1,cpg,3017624,CSC2_3B,1
5,1,cpg,3019021,CSC2_3B,1
7,1,cpg,3020724,CSC2_3B,1


In [36]:
df[0].to_hdf('data.h5', 'train')
df[1].to_hdf('data.h5', 'test')
df[2].to_hdf('data.h5', 'eval')

# Classification 

In [66]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import sklearn.metrics as met
import sklearn.base as skb

In [67]:
def feature_table(d, exclude=['knn_dist']):
    if exclude is not None:
        h = d.feature.str.startswith(exclude[0])
        for e in exclude[1:]:
            h |= d.feature.str.startswith(e)    
        d = d.loc[~h]
    d = pd.pivot_table(d, index=['chromo', 'pos'], columns=['sample', 'feature'], values='value')
    return d

def split_Xy(d, sample):
    h = ~d[(sample, 'cpg')].isnull()
    d = d.loc[h]
    d = d[[x for x in d.columns if x[0] == sample or not x[1].startswith('cpg')]]
    d = d.dropna()
    y = d[(sample, 'cpg')]
    x = d[[x for x in d.columns if x != (sample, 'cpg')]]
    return (x, y)

def split_XY(d):
    is_y = np.array([x[1] == 'cpg' for x in d.columns], dtype='bool')
    Y = d.loc[:, is_y]
    X = d.loc[:, ~is_y]
    h = ~X.isnull().any(axis=1)
    X = X.loc[h]
    Y = Y.loc[h]
    Y.columns = Y.columns.get_level_values(0)
    return (X, Y)
    
class MultitaskClassifier(object):
    
    def __init__(self, m):
        self.model = m
        
    def Xy_(self, X, Y, task):
        y = Y[:, task]
        h = ~np.isnan(y)
        X = X[h]
        y = y[h]
        return (X, y)
    
    def fit(self, X, Y):
        X = np.asarray(X)
        Y = np.asarray(Y)
        self.num_tasks = Y.shape[1]
        self.models = []
        for task in range(self.num_tasks):
            Xt, yt = self.Xy_(X, Y, task)
            m = skb.clone(self.model)
            m.fit(Xt, yt)
            self.models.append(m)
        
    def predict(self, X):
        X = np.asarray(X)
        Y = np.empty((X.shape[0], self.num_tasks))
        for task in range(self.num_tasks):
            Y[:, task] = self.models[task].predict(X)
        return Y
    
    def predict_proba(self, X):
        X = np.asarray(X)
        Y = np.empty((X.shape[0], self.num_tasks, 2))
        for task in range(self.num_tasks):
            Y[:, task] = self.models[task].predict_proba(X)
        return Y  
    
def complete_cases(x, y=None):
    x = [x]
    if y is not None:
        x.append(y)
    x = [np.asarray(x_) for x_ in x]
    h = None
    for x_ in x:
        if len(x_.shape) == 1:
            hx = ~np.isnan(x_)
        else:
            hx = ~np.any(np.isnan(x_), axis=1)
        if h is None:
            h = hx
        else:
            h &= hx
    xc = [x_[h] for x_ in x]
    return xc
                
def score(Y, Yp, fun=met.roc_auc_score):
    y = np.asarray(Y).ravel()
    yp = np.asarray(Yp).ravel()
    y, yp = complete_cases(y, yp)
    return fun(y, yp)

def scores(Y, Yp, fun=met.roc_auc_score):
    Y = np.asarray(Y)
    Yp = np.asarray(Yp)
    num_tasks = Y.shape[1]
    scores = []
    for task in range(num_tasks):
        y = Y[:, task]
        yp = Yp[:, task] 
        y, yp = complete_cases(y, yp)
        scores.append(fun(y, yp))
    return scores

def to_data_frame(x, y):
    assert np.all(x.shape == y.shape)
    x = pd.DataFrame(x)
    x.index = y.index
    x.columns = y.columns
    return x

In [39]:
df = [pd.read_hdf('data.h5', x) for x in ['train', 'test']]
df = [feature_table(x) for x in df]

In [41]:
m = MultitaskClassifier(RandomForestClassifier(max_depth=10))
X, Y = split_XY(df[0])
m.fit(X, Y)

In [47]:
YY = m.predict_proba(X)[:, :, 1]

In [48]:
scores(Y, YY)

[0.91810242102893502, 0.89371233721821164, 0.80348348084557109]

In [49]:
score(Y, YY)

0.89133986842401292

In [50]:
X, Y = split_XY(df[1])

In [51]:
YY = m.predict_proba(X)[:, :, 1]

In [52]:
scores(Y, YY)

[0.90349002637890696, 0.8802531539363152, 0.77811938320487928]

In [53]:
score(Y, YY)

0.8767470504973518

In [54]:
YY = to_data_frame(YY, Y)
X.to_hdf('predict.h5', 'X')
Y.to_hdf('predict.h5', 'Y')
YY.to_hdf('predict.h5', 'YP')

# Evaluation 

In [86]:
def rolling_apply(d, delta, fun, *args, **kwargs):
    rv = None
    for i in range(d.shape[0]):
        p = d.index[i]
        l = i
        while l > 0 and abs(d.index[l - 1] - p) <= delta:
            l -= 1
        r = i
        while r < d.shape[0] - 1 and abs(d.index[r + 1] - p) <= delta:
            r += 1
        di = d.iloc[l:(r + 1)]
        rvi = np.atleast_1d(fun(di, *args, **kwargs))
        if rv is None:
            rv = np.empty((d.shape[0], rvi.shape[0]))
        rv[i] = rvi
    rv = pd.DataFrame(rv, index=d.index)
    return rv

def cpg_cov(x, mean=False):
    h = np.mean(~np.isnan(x), axis=1)
    if mean:
        h = h.mean()
    return h

def cpg_density(x, c=1):
    return x.shape[0] / c

def var(x):
    x = np.ma.masked_array(x, np.isnan(x))
    return x.mean(axis=0).var()

def min_dist(x):
    """x is position vector"""
    rv = np.empty(len(x))
    rv.fill(np.nan)
    for i in range(len(x)):
        h = []
        if i > 0:
            h.append(abs(x[i] - x[i - 1]))
        if i < len(x) - 1:
            h.append(abs(x[i] - x[i + 1]))
        if len(h) > 0:
            rv[i] = min(h)
    return rv

def min_dist_nn(x, fun=np.mean):
    h = []
    for i in range(x.shape[1]):
        xi = x.iloc[:, i].dropna()
        h.append(pd.DataFrame(min_dist(xi.index), index=xi.index))
    h = pd.concat(h, axis=1)
    h.columns = x.columns
    assert np.all(h.shape == x.shape)
    if fun is not None:
        h = np.ma.masked_array(h, np.isnan(h))
        h = fun(h, axis=1).data
        h[h == 0] = np.nan
    return h

def stats(x, delta=1500):
    s = []
    s.append(cpg_cov(x))
    s.append(min_dist_nn(x))
    s.append(rolling_apply(x, delta, cpg_cov, mean=True))
    s.append(rolling_apply(x, delta, cpg_density, c=(2 * delta + 1)))
    s.append(rolling_apply(x, delta, var))
    s = [pd.DataFrame(s_, index=x.index) for s_ in s]
    s = pd.concat(s, axis=1)
    s.columns = ['cpg_cov', 'min_dist', 'win_cpg_cov', 'win_cpg_density', 'win_var']
    assert s.shape[0] == x.shape[0]
    return s

def group_apply(d, by, fun, level=False, *args, **kwargs):
    if level:
        g = d.groupby(level=by)
    else:
        g = d.groupby(by)
    r_all = []
    for k in g.groups.keys():
        dg = g.get_group(k)
        r = fun(dg, *args, **kwargs)
        if type(by) is list:
            for i in range(len(by)):
                r[by[i]] = k[i]
        else:
            r[by] = k
        r = r.set_index(by, append=True)
        r = r.swaplevel(0, r.index.nlevels-1)
        r_all.append(r)
    r_all = pd.concat(r_all)
    return r_all

def stats_all(d, *args, **kwargs):
    def set_index(d):
        d.index = d.index.get_level_values(1)
        return d
    return group_apply(d, 'chromo', lambda x: stats(set_index(x)), level=True)

def export_R(X, Y, Z, S, filename='eval'):
    X = X.copy()
    X.columns = ['_'.join(x) for x in X.columns.values]
    d = dict(X=X, Y=Y, Z=Z, S=S)
    for k, v in d.items():
        v = v.reset_index()
        fn = '%s_%s.csv' % (filename, k)
        v.to_csv(fn, index=False)

In [64]:
X = pd.read_hdf('predict.h5', 'X')
Y = pd.read_hdf('predict.h5', 'Y')
Z = pd.read_hdf('predict.h5', 'YP')

In [182]:
S = stats_all(Y)

In [214]:
export_R(X, Y, Z, S)

In [141]:
from rpy2.robjects import r
import pandas.rpy.common as com

def join_index(index, sep='_'):
    return [sep.join(x) for x in index.values]

def to_r(d):
    if d.columns.nlevels > 1:
        d = d.copy()
        d.columns = join_index(d.columns)
    d = d.reset_index()
    d = com.convert_to_r_dataframe(d)
    convert = r("""
    function(df) {
        data.frame(lapply(df, function(X) {
            if ("AsIs" %in% class(X))
                class(X) <- class(X)[-match("AsIs", class(X))]
            X
        }))
    }
""")
    d = convert(d)
    return d

def to_rds(d, filename):
    if type(d) is dict:
        r('d <- list()')
        for k, v in d.items():
            r.assign('dd', to_r(v))
            r('d$%s <- dd' % (k))
    else:
        r.assign('d', to_r(d))
    r("saveRDS(d, '%s')" % (filename))

In [143]:
to_rds({'X': X.head(), 'Y': Y.head(), 'Z': Z.head(), 'S': X.head()}, 'test.rds')